## Main API for this Bot

1. Get Block Number by Timestamp:
https://api.etherscan.io/api
   ?module=block
   &action=getblocknobytime
   &timestamp=1578638524
   &closest=before
   &apikey=YourApiKeyToken
2. Get a list of 'Normal' Transactions By Address:
    https://api.etherscan.io/api
   ?module=account
   &action=txlist
   &address=0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC
   &startblock=0
   &endblock=99999999
   &page=1
   &offset=10
   &sort=asc
   &apikey=YourApiKeyToken
3. Get Ether Balance for a Single Address
    https://api.etherscan.io/api
   ?module=account
   &action=balance
   &address=0xde0b295669a9fd93d5f28d9ec85e40f4cb697bae
   &tag=latest
   &apikey=YourApiKeyToken
4. Get a list of 'ERC20 - Token Transfer Events' by Address
    https://api.etherscan.io/api
   ?module=account
   &action=tokentx
   &contractaddress=0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
   &address=0x4e83362442b8d1bec281594cea3050c8eb01311c
   &page=1
   &offset=100
   &startblock=0
   &endblock=27025780
   &sort=asc
   &apikey=YourApiKeyToken

## Library

In [36]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
import hashlib
import hmac
import base64
import random
from urllib.parse import urlencode, quote_plus
import http.client
import json

## Gobal env

In [37]:
api_key_list = ['BJJKDZ5R2KFBZW4GV1MXEPRRU1EJVN2K1G', 'PJ8X4ADB1VBVAQNHTXW13J7TCB46X7THT7', 'F2GDTZ2N53MDKFZ8CSGQR4Y2AHZZTJZH6M']
base_url = 'https://api.etherscan.io/api'
X_BLOBR_KEY_list = ['j0QM4RJ9PnP5r1J3SnkYBQyVOts7m2tN', 'EElaWmMD94plCowVHeYXRRkqb27WqyXH']
dex_tool_base_url = 'https://open-api.dextools.io/free'

# helper method that works for all the above api

In [38]:
def request_helper(module, action, contractaddress, address, timestamp, startblock, endblock):
    params = {
    'module': module,
    'action': action,
    'address': address,
    'contractaddress': contractaddress,
    'timestamp': timestamp,
    'startblock': startblock,
    'endblock': endblock,
    'closest': 'before',
    'tag': 'latest',
    'apikey': '',
    'page': 1,
    'offset': 20
}
    id = random.randint(1, 9633)
    id = id % len(api_key_list) 
    params['apikey'] = api_key_list[id]
    response = requests.get(url=base_url, params=params)
    return response.json()

In [ ]:
def get_cur_timestamp():
        # Get the current time in UTC+8
    current_time = datetime.now()
    # Calculate the start time for the last 60 minutes
    start_time = current_time
    
    # Convert the start time to a timestamp in milliseconds
    active_timeStamp = int(start_time.timestamp())
    
    utc_time = datetime.utcfromtimestamp(active_timeStamp / 1000) + timedelta(hours=8)
    date_str = utc_time.strftime('%Y-%m-%d %H:%M:%S')
    print(date_str)
    print(active_timeStamp)
    return active_timeStamp
get_cur_timestamp()

## get the ERC-20 token's price

In [ ]:
def get_token_price(address):
    chain = 'ether'
    url = f'/v2/token/{chain}/{address}/price'
    # getting the token's price 
    headers = {
    "X-BLOBR-KEY": "j0QM4RJ9PnP5r1J3SnkYBQyVOts7m2tN"
    }
    response = requests.get(url=dex_tool_base_url+url, headers=headers)
    data = response.json()
    return data

## Current block number of ethereum

In [ ]:
def get_cur_block():
    curTimeStamp = get_cur_timestamp()
    data = request_helper(module='block', action='getblocknobytime', timestamp=curTimeStamp, contractaddress=None, address=None, endblock='',startblock='')
    return data['result']
get_cur_block()

In [75]:
def convertToEth(token_value, decimal):
    eth_value = token_value / (10 ** decimal)
    return eth_value

In [76]:
data = request_helper(module='account', action='tokentx', address='0x465fcc902f7b26f1d92c02cdfa384006a1b4dae9', contractaddress=None, startblock='19017054', endblock=get_cur_block(), timestamp='')
token_value = int(data['result'][0]['value'])
decimal = int(data['result'][0]['tokenDecimal'])

eth_value = convertToEth(token_value, decimal)
print("Value in ETH:", eth_value)

1970-01-21 01:43:04
1705384751
Value in ETH: 15000000000.0


In [99]:
(20000000000000000000) / 10 ** 9

20000000000.0

In [100]:
20000000000.0 * 0.0000000268

536.0

In [6]:
eth = int(data['result']) / 10**18

In [7]:
eth

2.2931494564740245

{'statusCode': 200,
 'data': {'price': 1.2646463580791198e-07,
  'priceChain': 4.997921126579462e-11,
  'price5m': 9.857158746943813e-08,
  'priceChain5m': 3.895579316278381e-11,
  'variation5m': 28.297249800427558,
  'variationChain5m': 28.297249800427558,
  'price1h': 4.293049671599591e-09,
  'priceChain1h': 1.6932366817251034e-12,
  'variation1h': 2845.7995010989757,
  'variationChain1h': 2851.6967004798644,
  'price6h': 1.688754243545069e-08,
  'priceChain6h': 6.660674462972156e-12,
  'variation6h': 648.8634672054752,
  'variationChain6h': 650.362617834541,
  'price24h': 1.688754243545069e-08,
  'priceChain24h': 6.660674462972156e-12,
  'variation24h': 648.8634672054752,
  'variationChain24h': 650.362617834541}}

In [48]:
df = pd.read_csv("./smart_traders.csv")

In [ ]:
df.columns = ['altcoin_smartWallet']
wallet_list = df['altcoin_smartWallet'].to_list()